In [1]:
import pandas as pd
from collections import defaultdict
from collections import Counter
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
import numpy as np


In [2]:
%matplotlib inline  
%autosave 300
pd.options.display.max_columns = None
feature_names =defaultdict(lambda:'None')

Autosaving every 300 seconds


# Classification based on published Data:

Risk High/low set at 38.40 Readiness high/low set at 46.51


In [3]:
def categorize(row):
    if row['overall_risk']== 1 and row['overall_readiness']==1:
        return 4
    elif row['overall_risk']== 1 and row['overall_readiness']==0:
        return 3
    elif row['overall_readiness'] == 1:
        return 2
    else:
        return 1
    
risk_ready=pd.read_csv('data/Overall Risk & Readiness Scores.csv')

risk_ready_old=risk_ready.columns
new_names=['city', 'state', 'geo_id','overall_risk', 'overall_readiness']
risk_ready.columns = new_names
    
# Puerto Rico is being dropped at rows [19, 35, 193, 220] because of data inconsistency
risk_ready.drop([19, 35, 193, 220], inplace = True)
risk_ready['overall_risk'] = np.where(risk_ready['overall_risk']>=.3840, 1, 0)
risk_ready['overall_readiness'] = np.where(risk_ready['overall_readiness']>=.4651, 1, 0)
risk_ready['risk_ready_cat']=risk_ready.apply(lambda row: categorize(row), axis=1)

risk_ready_dict = dict(zip(new_names, risk_ready_old))
feature_names['risk_ready']=risk_ready_dict

Clean main data File

In [6]:
city_ind = pd.read_csv('data/City Indicators.csv')     


city_ind_old=city_ind.columns
new_names=['city','state','geo_id','GDP_water_all','perc_fff','d_plan','w_plan','base_w_stress','invest','tax_break','corruption','perc_vote','innovate','climate_real','hs_ed','pop_dens','w_quality','debt','flood_pop','flood_build','flood_car','impervious','beds_1000','alone_65','child_5','health_ins','older_1999','mobile_home','rent_50plus','trees','heating','disabled','poverty','older_1979','work_outside','no_car','pop_under_1ft','pop_under_3ft','impact_plus1ft','impact_plus3ft', 'col_empty','lat','long','county','region','city_km2','pop_2010','pop_2011','pop_2012','pop_2013','pop_2014','pop_2015','median_income']
city_ind.columns = new_names

city_ind.drop(columns =['col_empty'], inplace= True)
indicators_dict = dict(zip(city_ind.columns, city_ind_old))



#Puerto Rico is being dropped at rows [19, 35, 193, 220] because of data inconsistency
#city_ind.drop([19, 35, 193, 220], inplace = True)
#two cities had their order of magnitude incorrect for % of people in floodzone. Their neighboring values match the lower magnitude'
city_ind=city_ind.replace(412.9208681,41.29208681)
city_ind=city_ind.replace(101.6959823,10.16959823)
city_ind.replace('Bayam<U+0086>n','Bayamon')
city_ind['risk_ready_cat']= risk_ready['risk_ready_cat']

city_ind.drop(columns=['city','state','geo_id','county', 'region','pop_2010', 'pop_2011', 'pop_2012', 'pop_2013', 'pop_2014'], inplace=True)
city_ind['d_plan'] = city_ind['d_plan'].map({'Yes': 1, 'No': 0})
city_ind['w_plan'] = city_ind['w_plan'].map({'Yes': 1, 'No': 0})
city_ind.fillna(0, inplace=True)



In [7]:

trees_cat=city_ind['risk_ready_cat']
trees_data = city_ind.drop(columns=['risk_ready_cat'])
trees_data.columns


Index(['GDP_water_all', 'perc_fff', 'd_plan', 'w_plan', 'base_w_stress',
       'invest', 'tax_break', 'corruption', 'perc_vote', 'innovate',
       'climate_real', 'hs_ed', 'pop_dens', 'w_quality', 'debt', 'flood_pop',
       'flood_build', 'flood_car', 'impervious', 'beds_1000', 'alone_65',
       'child_5', 'health_ins', 'older_1999', 'mobile_home', 'rent_50plus',
       'trees', 'heating', 'disabled', 'poverty', 'older_1979', 'work_outside',
       'no_car', 'pop_under_1ft', 'pop_under_3ft', 'impact_plus1ft',
       'impact_plus3ft', 'lat', 'long', 'city_km2', 'pop_2015',
       'median_income'],
      dtype='object')

# Calculate Category variables

In [ ]:

# frame one (calc_from_raw) creates the six values for each hazard
calc_from_raw =pd.DataFrame()
calc_from_raw['cities']= cities

calc_from_raw['econ_c']=cleaned['debt']+cleaned['invest']+cleaned['tax_break']
calc_from_raw['gov_c']=cleaned['innovate']+cleaned['climate_real']
calc_from_raw['social_c']=cleaned['corruption']+cleaned['perc_vote']
#make sure to use flood_expos for flood. Use this one for the rest
calc_from_raw ['expos_c']=cleaned['pop_2015']

#heat - adaption and sensitivity formulas unique
calc_from_raw['heat_adapt_c']=cleaned['beds_1000']+cleaned['trees']
calc_from_raw['heat_sens_c']=cleaned['alone_65']+cleaned['poverty']+cleaned['disabled']



#cold - adaption and sensitivity formulas unique
calc_from_raw['cold_adapt_c']=cleaned['beds_1000']+cleaned['heating']
calc_from_raw['cold_sens_c']=cleaned['alone_65']+cleaned['work_outside']+cleaned['child_5']

#flood - exposure, sensitivity and adaption formulas unique
calc_from_raw['flood_expos_c']=cleaned['flood_pop']+cleaned['flood_build']
calc_from_raw['flood_sens_c']=cleaned['older_1999']+cleaned['mobile_home']
calc_from_raw['flood_adapt_c']=cleaned['beds_1000']+cleaned['w_quality']


#drought - sensitivity and adaption unique
calc_from_raw['drought_sens_c']=cleaned['GDP_water_all']+cleaned['perc_fff']
calc_from_raw['drought_adapt_c']=cleaned['d_plan']+cleaned['w_plan']
calc_from_raw.dropna(inplace=True)


# Calculate Summary Values for categories

In [ ]:
''' Study standardized then used standardized versions
next stage
'''
from sklearn.preprocessing import StandardScaler

prescale_table=calc_from_raw.drop(columns=['cities'])
prescale_features=prescale_table.columns
scaler=StandardScaler()
calc_hazard_stand=scaler.fit_transform(prescale_table)
calc_from_raw_scaled =pd.DataFrame(calc_hazard_stand, columns=prescale_features)
#calc_from_raw_scaled['cities']= cities


risk_recovery_calc=pd.DataFrame()
risk_recovery_calc['cities']= cities
#Readiness is the value for all hazards so it will only be done once
risk_recovery_calc['readiness_c']=(calc_from_raw_scaled['gov_c']+calc_from_raw_scaled['social_c']+calc_from_raw_scaled['econ_c'])/3

                                                                                                         
risk_recovery_calc['heat_risk']=(calc_from_raw_scaled['expos_c']+calc_from_raw_scaled['heat_adapt_c']+(1-calc_from_raw_scaled['heat_sens_c']))/3
risk_recovery_calc['cold_risk']=(calc_from_raw_scaled['expos_c']+calc_from_raw_scaled['cold_adapt_c']+(1-calc_from_raw_scaled['cold_sens_c']))/3
risk_recovery_calc['flood_risk']=(calc_from_raw_scaled['flood_expos_c']+calc_from_raw_scaled['flood_adapt_c']+(1-calc_from_raw_scaled['flood_sens_c']))/3
risk_recovery_calc['drought_risk']=(calc_from_raw_scaled['expos_c']+calc_from_raw_scaled['drought_adapt_c']+(1-calc_from_raw_scaled['drought_sens_c']))/3
risk_recovery_calc['risk_c']=risk_recovery_calc['heat_risk']+risk_recovery_calc['cold_risk']+risk_recovery_calc['flood_risk']     
risk_recovery_calc.dropna(inplace=True)